# Formulations Probabilistes

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px # Pour installer le module Plotly, exécuter la commande ci-dessous
import plotly.graph_objects as go
#from tabulate import tabulate

import random
import sys
import seaborn as sns
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE, RFECV, SelectKBest, f_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
IN_COLAB = 'google.colab' in sys.modules

import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 100)

## Indépendance Statistique

Deux variables aléatoires X et Y sont dites **indépendantes** si la réalisation de X n'a aucune influence sur la réalisation de Y, et vice versa. Cela se traduit mathématiquement par: <br>
***P(X=x, Y=y) = P(X=x)⋅P(Y=y) = P(X=x)⋅P(Y=y) = P(Y=y, X=x)*** &nbsp; $\forall \, (x,y) \in (X,Y)$

In [ ]:
from scipy.stats import multivariate_normal

# Génération des données 
mean = [0, 0] # Moyenne
cov = [[1, 0], [0, 1]] # Matrice de covariance pour varaiables indépendantes
x, y = np.random.multivariate_normal(mean, cov, 5000).T

# Affichage des variables indépendantes
plt.figure(figsize=(8, 6))
plt.scatter(x, y, alpha=0.5)
plt.title('Variables Indépendantes')
plt.xlabel('Variable X')
plt.ylabel('Variable Y')
plt.grid(True)
plt.show()

# Illustration de la distribution gaussienne 2D
x, y = np.mgrid[-3:3:.01, -3:3:.01]
pos = np.dstack((x, y))
rv = multivariate_normal(mean, cov)
plt.figure(figsize=(8, 6))
plt.contourf(x, y, rv.pdf(pos), cmap="inferno")
plt.title('Distribution Gaussienne 2D pour des Variables Indépendantes')
plt.xlabel('Variable X')
plt.ylabel('Variable Y')
plt.colorbar()
plt.show()


## Dépendance Statistique

Deux variables X et Y sont dites **dépendantes** si la connaissance de la valeur de X donne des informations sur la valeur de Y et vice versa. Cela signifie que la probabilité conjointe de X et Y n'est pas égale au produit des probabilités marginales; Autrement dit:
***P(X=x, Y=y)*** $\neq$ ***P(X=x)⋅P(Y=y)*** <br>


### Covariance

Pour quantifier la dépendance linéaire entre deux variables, on utilise souvent la **covariance** 
La covariance entre deux variables X et Y est définie comme :
 &nbsp; **Cov(X,Y)** = $\mathbb{E}$[(X−$\mathbb{E}$[X])(Y−$\mathbb{E}$[Y])] où $\mathbb{E}$ désigne l'espérance (ou moyenne). <br>
Si Cov(X,Y) = 0 (*Cas de l'exemple plus haut*), cela indique qu'il n'y a pas de relation linéaire entre X et Y. Cependant, cela ne signifie pas nécessairement que X et Y sont indépendants.

#### Si Cov(X,Y) > 0, cela indique une relation linéaire positive entre X et Y. (*Voir Figure ci-dessous*)

In [ ]:
# Génération des données
mean = [0, 0] # Moyenne
cov = [[1, 0.8], [0.8, 1]]  # Matrice de covariance pour variables positivement dépendantes
x, y = np.random.multivariate_normal(mean, cov, 5000).T

# Affichage des variables dépendantes avec covariance positive
plt.figure(figsize=(8, 6))
plt.scatter(x, y, alpha=0.5)
plt.title('Variables Dépendantes avec Covariance Positive')
plt.xlabel('Variable X')
plt.ylabel('Variable Y')
plt.grid(True)
plt.show()

# Illustration de la distribution gaussienne 2D
x, y = np.mgrid[-3:3:.01, -3:3:.01]
pos = np.dstack((x, y))
rv = multivariate_normal(mean, cov)
plt.figure(figsize=(8, 6))
plt.contourf(x, y, rv.pdf(pos), cmap="viridis")
plt.title('Distribution Gaussienne 2D pour des Variables Dépendantes avec Covariance Positive')
plt.xlabel('Variable X')
plt.ylabel('Variable Y')
plt.colorbar()
plt.show()


#### Si Cov(X,Y) < 0, cela indique une relation linéaire négative entre X et Y. (*Voir Figure ci-dessous*)

In [ ]:
# Paramètres pour les variables dépendantes avec covariance négative
mean = [0, 0]
cov = [[2, -0.8], [-0.8, 1]]  

# Génération des données
x, y = np.random.multivariate_normal(mean, cov, 5000).T

# Affichage des variables dépendantes avec covariance négative
plt.figure(figsize=(8, 6))
plt.scatter(x, y, alpha=0.5)
plt.title('Variables Dépendantes avec Covariance Négative')
plt.xlabel('Variable X')
plt.ylabel('Variable Y')
plt.grid(True)
plt.show()

# Illustration de la distribution gaussienne 2D
x, y = np.mgrid[-3:3:.01, -3:3:.01]
pos = np.dstack((x, y))
rv = multivariate_normal(mean, cov)
plt.figure(figsize=(8, 6))
plt.contourf(x, y, rv.pdf(pos), cmap="viridis")
plt.title('Distribution Gaussienne 2D pour des Variables Dépendantes avec Covariance Négative')
plt.xlabel('Variable X')
plt.ylabel('Variable Y')
plt.colorbar()
plt.show()


## Distribtions Jointes

Une distribution jointe décrit la manière dont deux ou plusieurs variables aléatoires se distribuent simultanément. En d'autres termes, elle montre comment les valeurs de plusieurs variables sont liées entre elles et comment elles varient ensemble. Elle fournit des informations sur la probabilitéou la densité de différentes combinaisons de valeurs pour plusieurs variables. Elle est souvent notée **P(X, Y)**.

### Marginalisation

La distribution marginale de X est obtenue en **marginalisant** Y hors de la distribution jointe. La marginalisation est une technique utilisée pour simplifier une distribution de probabilité multivariée en intégrant une ou plusieurs variables pour obtenir la distribution marginale des variables restantes. Cela est particulièrement utile lorsque l'on souhaite se concentrer sur un sous-ensemble des variables.<br>

Pour une variable continue: P(X=x) =  $\int_{-\infty}^{\infty} P(X=x, Y=y) \, dy$ <br>
Pour une variable continue: P(X=x) =  $\sum_y{P(X=x, Y=y)}$ <br>

Cela peut être observé avec les deux figures qui suivent

In [ ]:
np.random.seed(42)
x = 4*np.random.randn(1000)
y = np.random.randn(1000) + 3

# Création du DataFrame
data = pd.DataFrame({
    'Variable_X': x,
    'Variable_Y': y,  
})


In [ ]:
# Création de la joint plot avec des couleurs et des repères
sns.set(style="white", color_codes=True)
g = sns.jointplot(x='Variable_X', y='Variable_Y', data=data, kind='kde', fill=True, cmap="viridis")

# Ajout de la moyenne
mean_x = data['Variable_X'].mean()
mean_y = data['Variable_Y'].mean()
g.ax_joint.scatter(mean_x, mean_y, color='red', label='Mean', s=100, marker='x')

# Ajouter les labels et la légende
g.ax_joint.set_xlabel('X')
g.ax_joint.set_ylabel('Y')
g.ax_joint.legend()

# Ajouter la colorbar
cbar = g.fig.colorbar(g.ax_joint.collections[0], ax=g.ax_joint, orientation='vertical')
#cbar.set_label('Densité')
plt.subplots_adjust(right=1.5)
g.fig.suptitle('Porbabilité Jointe en 2D', x=0, y=1.05, ha='left')

# Affichage de la figure
plt.show()


La partie au centre sont des lignes de niveaux et représentent la densité de probabilité jointe des deux distributions $X \, et \,Y$. 
<br>
Les graphiques en haut et à droite représente respectivement aux distributions de probabilité marginales de la variable $X$ et de la variable $Y$ .

In [ ]:
# Calcul des densités de probabilité
x = np.linspace(data['Variable_X'].min(), data['Variable_X'].max(), 100)
y = np.linspace(data['Variable_Y'].min(), data['Variable_Y'].max(), 100)
X, Y = np.meshgrid(x, y)

# Estimation de la densité
from scipy.stats import gaussian_kde
kde = gaussian_kde([data['Variable_X'], data['Variable_Y']])
Z = kde(np.vstack([X.ravel(), Y.ravel()]))
Z = Z.reshape(X.shape)

# Création du graphique en 3D
fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='Viridis', colorbar=dict(title='Densité'))])

# Ajouter des labels et un titre
fig.update_layout(
    title='Distribution Jointe en 3D',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Densité'
    ),
    width=1000,  # Largeur du graphique en pixels
    height=800   # Hauteur du graphique en pixels
)
# Afficher la figure
fig.show()

### Distribution jointe, conditionnelle et évidence
Pour illustrer ces trois concepts, nous reprendrons l'exemple des résultats à un examen de statistique par des élèves issus du programme the **math** et du programme d'**informatique**.  Les données de cette expérience sont dans le fichier *exam_stat.csv*

In [ ]:
# Création du DataFrame
np.random.seed(42) 
cursus = np.random.choice(np.array(['Math', 'Info']), 100)
notes = np.random.choice(np.arange(8, 17), 100)
resultats = pd.DataFrame({
    'Cursus': cursus,
    'Notes': notes,  
})

# Affichage & Sauvegarde
#print(tabulate(data.head(5), headers='keys', tablefmt='grid', showindex=False))
resultats.to_csv('exam_stat.csv', index=False)

In [ ]:
# Création du DataFrame
np.random.seed(42)

nbr_math = 35
nbr_info = 65

cursus_math = ['Math'] * nbr_math
cursus_info = ['Info'] * nbr_info
cursus = np.concatenate([cursus_math, cursus_info])


notes_math = np.random.choice(np.arange(13, 18), nbr_math) #Plage plus elevée
notes_info = np.random.choice(np.arange(8, 15), nbr_info) #Plage plus basse
notes = np.concatenate([notes_math, notes_info])

resultats = pd.DataFrame({
    'Cursus': cursus,
    'Notes': notes
})

# Affichage & Sauvegarde
resultats.to_csv('dataset.csv', index=False)


#### Jointes et Evidence
affichage graphique de la distribution "Evidence" $P(x)$ et des 2 distributions jointes $P(x,math)$ et $P(x,info)$.

In [ ]:
from scipy import stats

# Données initiales
x1 = resultats[resultats['Cursus'] == 'Math']['Notes'].to_numpy()
x2 = resultats[resultats['Cursus'] == 'Info']['Notes'].to_numpy()
x_tot = np.append(x1, x2)

prop1 = x1.size / (x1.size + x2.size)
prop2 = x2.size / (x1.size + x2.size)

kde1 = stats.gaussian_kde(x1, bw_method=0.5)
kde2 = stats.gaussian_kde(x2, bw_method=0.5)
kde_tot = stats.gaussian_kde(x_tot, bw_method=0.5)

fig, ax = plt.subplots(figsize=(12, 8))

x_eval = np.linspace(0, 20, num=200)
ax.plot(x_eval, kde1(x_eval) * prop1, 'k-', label="P(x, cursus=Math)")
ax.plot(x_eval, kde2(x_eval) * prop2, 'r-', label="P(x, cursus=Info)")
ax.plot(x_eval, kde2(x_eval) * prop2 + kde1(x_eval) * prop1 + 0.002, 'b-', label="P(x)")

# Affichage du graphique
ax.legend()
ax.set_title('Distributions jointes et Evidence')
plt.show()


Partant des résultats à l'examen final du cours de stats de $100$ élèves et sachant que ces derniers sont issus de deux programmes différents, le but  est de déterminr à partir d'une note quelconque, le cursus auquel appartient l'élève ayant obtenu cette note.
Le graphique ci-dessus présente les distributions marginales  et de probabilité de la variable **notes**, par rapport aux deux valeurs possibles de la variable **cursus**(*Math* & *Info*) 

In [ ]:
# Données initiales
x1 = resultats[resultats['Cursus'] == 'Math']['Notes'].to_numpy()
x2 = resultats[resultats['Cursus'] == 'Info']['Notes'].to_numpy()
x_tot = np.append(x1, x2)

prop1 = x1.size / (x1.size + x2.size)
prop2 = x2.size / (x1.size + x2.size)

kde1 = stats.gaussian_kde(x1, bw_method=0.5)
kde2 = stats.gaussian_kde(x2, bw_method=0.5)
kde_tot = stats.gaussian_kde(x_tot, bw_method=0.5)

fig, ax = plt.subplots(figsize=(12, 8))

x_eval = np.linspace(0, 20, num=200)
ax.plot(x_eval, kde1(x_eval) * prop1, 'k-', label="P(x, cursus=Math)")
ax.plot(x_eval, kde2(x_eval) * prop2, 'r-', label="P(x, cursus=Info)")
ax.plot(x_eval, kde2(x_eval) * prop2 + kde1(x_eval) * prop1 + 0.002, 'b-', label="P(x)")

# Note à projeter
note = 13

# Calcul des densités à la note 13
density_math = kde1(note) * prop1
density_info = kde2(note) * prop2

# Projeter les points sur les courbes
ax.plot(note, density_math, 'ko', label=f'Projection sur Math (Note = {note})')
ax.plot(note, density_info, 'ro', label=f'Projection sur Info (Note = {note})')
ax.axvline(note, color='gray', linestyle='--')  

# Annotation des densités
density_math_value = density_math.item()
density_info_value = density_info.item()
ax.text(note + 0.2, density_math_value, f'{density_math_value:.3f}', ha='left', va='bottom', color='black')
ax.text(note + 0.2, density_info_value, f'{density_info_value:.3f}', ha='left', va='bottom', color='black')

# Légende et titre
ax.legend()
ax.set_title('Projection de la note 13')

# Affichage du graphique
plt.show()


On projecte sur les deux courbes de distributions jointes la valeur ***note = 13***. On obtient: $P(note=13, info) = 0.092$ et $P(note=13, math) = 0.035$ 
<br> 
$P(note=13, info) > P(note=13, math)$.  
<br>
La prédiction à l'aide d'un maximum à priori indique que l'élève ayant obtenu la note **13** appratient à la classe **Info**

#### Conditionnelles & Evidence

In [ ]:
# Création de la distribution conditionnelle pour chaque cursus
sns.set(style="white", color_codes=True)

# KDE Plot pour le cursus 'Math'
plt.figure(figsize=(12, 6))
sns.kdeplot(data=resultats[resultats['Cursus'] == 'Math']['Notes'], label='P(Notes | Cursus=Math)', fill=True, cmap="Blues")

# KDE Plot pour le cursus 'Info'
sns.kdeplot(data=resultats[resultats['Cursus'] == 'Info']['Notes'], label='P(Notes | Cursus=Info)', fill=True, cmap="Reds")

sns.kdeplot(resultats['Notes'], color='red', label='P(Notes)')

# Ajouter les labels et la légende
plt.xlabel('Notes')
plt.ylabel('Densité')
plt.title('Distributions Conditionnelles et Evidence')
plt.legend()

# Affichage du graphique
plt.show()


La figure ci dessus utilise le même contexte utilisé plus haut et présente un graphique comparatif des courbes de densité de probabilité et conditionnelle de la variable **notes** sachant une réalisation de la variable **cursus**. On remarque que l'amplitude des courbes conditonnelles est plus elevés que celle des courbe de densité globale. Cela est du au fait que pour les conditionnelles, les densités sont calculées sur un sous ensemble des élèves sur lesquelles les fréquences des notes sont plus élevées.